In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

# Problem statement

Основная задача: Бинарная классификация
* 0 – клиент не открыл кредитный счет в банке
* 1 - клиент открыл счет в кредитном банке

Необходимо классифицировать клиентов на тех кто не откроет кредитного счета, и тех кто его откроет
Данные:
+ credit_train.csv - Датасет содержит данные о кредитах на покупку электроники, которые были одобрены 

1. client_id - идентификационный признак клиента
2. gender - пол клиента.
3. age - возраст клиента
4. marital_status - семейный статус клиента
5. job_position - сфера деятельности
6. credit_sum - сумма кредита (замените запятые на точки)
7. credit_month - срок кредита
8. tariff_id - тарифный план обслуживания в банке
9. score_shk - внутренняя (замените запятые на точки)
10. education - степень образованности
11. living_region - регионом проживания
12. monthly_income - ежемесяный доход
13. credit_count - кол-во кредитов на момент подачи заявки
14. open_account_flg - факт открытия кредитного счета в данном банке
________________________________________


# Read data

In [2]:
test_df=pd.read_csv(r'Data\Raw\credit_test.csv', sep=';', encoding='windows-1251', decimal=',')
train_df=pd.read_csv(r'Data\Raw\credit_train.csv', sep=';', encoding='windows-1251', decimal=',')

# Exploratory Analysis and Data clearing

In [3]:
train_df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,59998.00,10,1.6,0.770249,GRD,КРАСНОДАРСКИЙ КРАЙ,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,10889.00,6,1.1,0.248514,GRD,МОСКВА,43000.0,2.0,0.0,0
2,3,M,32,MAR,SPC,10728.00,12,1.1,0.459589,SCH,ОБЛ САРАТОВСКАЯ,23000.0,5.0,0.0,0
3,4,F,27,DIV,SPC,12009.09,12,1.1,0.362536,GRD,ОБЛ ВОЛГОГРАДСКАЯ,17000.0,2.0,0.0,0
4,5,M,45,MAR,SPC,16908.89,10,1.1,0.421385,SCH,ЧЕЛЯБИНСКАЯ ОБЛАСТЬ,25000.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170741,170742,F,27,UNM,SPC,64867.00,12,1.1,0.535257,GRD,РЕСПУБЛИКА ТАТАРСТАН,40000.0,6.0,0.0,0
170742,170743,F,24,MAR,SPC,17640.00,6,1.6,0.573287,SCH,САНКТ-ПЕТЕРБУРГ Г,30000.0,1.0,0.0,0
170743,170744,F,31,UNM,SPC,27556.47,10,1.32,0.416098,GRD,ПРИМОРСКИЙ КРАЙ,40000.0,1.0,0.0,0
170744,170745,F,53,DIV,PNA,6189.00,12,1.1,0.482595,SCH,ПЕНЗЕНСКАЯ ОБЛ,31000.0,2.0,0.0,0


In [4]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91940 entries, 0 to 91939
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   client_id             91940 non-null  int64  
 1   gender                91940 non-null  object 
 2   age                   91940 non-null  int64  
 3   marital_status        91940 non-null  object 
 4   job_position          91940 non-null  object 
 5   credit_sum            91940 non-null  float64
 6   credit_month          91940 non-null  int64  
 7   tariff_id             91940 non-null  object 
 8   score_shk             91940 non-null  float64
 9   education             91940 non-null  object 
 10  living_region         91824 non-null  object 
 11  monthly_income        91940 non-null  int64  
 12  credit_count          87237 non-null  float64
 13  overdue_credit_count  87237 non-null  float64
dtypes: float64(4), int64(4), object(6)
memory usage: 9.8+ MB


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170746 entries, 0 to 170745
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   client_id             170746 non-null  int64  
 1   gender                170746 non-null  object 
 2   age                   170746 non-null  int64  
 3   marital_status        170746 non-null  object 
 4   job_position          170746 non-null  object 
 5   credit_sum            170746 non-null  float64
 6   credit_month          170746 non-null  int64  
 7   tariff_id             170746 non-null  object 
 8   score_shk             170746 non-null  float64
 9   education             170746 non-null  object 
 10  living_region         170554 non-null  object 
 11  monthly_income        170745 non-null  float64
 12  credit_count          161516 non-null  float64
 13  overdue_credit_count  161516 non-null  float64
 14  open_account_flg      170746 non-null  int64  
dtype

Преобразовываю типы данных в соответствии с их интерпретацией.

In [6]:
test_df=test_df.astype({'tariff_id': 'float64'})
train_df=train_df.astype({'tariff_id': 'float64'})

category_list=['gender', 'age', 'marital_status', 'job_position', 'credit_month',
               'tariff_id', 'education', 'living_region', 'credit_count', 'overdue_credit_count', 'open_account_flg']

category_dict=dict()
for cat in category_list[:-1]:
    category_dict[cat]='category'
test_df=test_df.astype(category_dict)

category_dict=dict()
for cat in category_list:
    category_dict[cat]='category'
train_df=train_df.astype(category_dict)

В данных наблюдаются пропущенные значения, при этом модальность данных не представленна в виде временного ряда. Данные выглядят как реальные данные задачи кредитного скоринга, что позволяет предположить отсутствие известных заранее статистических зависимостей. Перечисленные причины не позволяют использовать существующие методы восстановления данных, следовательно строки с пропущенными данные должны быть удалены.

* Вывод: Необходимо удалить строки с пропущенными значениями.

## Drop NaN valies

In [7]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 87130 entries, 0 to 91939
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   client_id             87130 non-null  int64   
 1   gender                87130 non-null  category
 2   age                   87130 non-null  category
 3   marital_status        87130 non-null  category
 4   job_position          87130 non-null  category
 5   credit_sum            87130 non-null  float64 
 6   credit_month          87130 non-null  category
 7   tariff_id             87130 non-null  category
 8   score_shk             87130 non-null  float64 
 9   education             87130 non-null  category
 10  living_region         87130 non-null  category
 11  monthly_income        87130 non-null  int64   
 12  credit_count          87130 non-null  category
 13  overdue_credit_count  87130 non-null  category
dtypes: category(10), float64(2), int64(2)
memory usage: 4.3 MB


In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 161341 entries, 0 to 170745
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   client_id             161341 non-null  int64   
 1   gender                161341 non-null  category
 2   age                   161341 non-null  category
 3   marital_status        161341 non-null  category
 4   job_position          161341 non-null  category
 5   credit_sum            161341 non-null  float64 
 6   credit_month          161341 non-null  category
 7   tariff_id             161341 non-null  category
 8   score_shk             161341 non-null  float64 
 9   education             161341 non-null  category
 10  living_region         161341 non-null  category
 11  monthly_income        161341 non-null  float64 
 12  credit_count          161341 non-null  category
 13  overdue_credit_count  161341 non-null  category
 14  open_account_flg      161341 non-null  ca

* Необходимо удалить дубликаты в данных

## Find and drop dublicates

In [9]:
set(test_df.duplicated()), set(train_df.duplicated())

({False}, {False})

Полностью дублирующиеся строки отсутствуют. 
* Необходимо выявить наличие дублирующихся клиентских ID.

In [10]:
set(test_df.duplicated(subset=['client_id'])), set(train_df.duplicated(subset=['client_id']))

({False}, {False})

Дублирующиеся клиентские ID отсутствуют.
* Необходимо выявить наличие выбросов в данных

## Finding Outliers and Anomalies

In [11]:
train_df.describe()

,client_id,credit_sum,score_shk,monthly_income
count,161341.000000,161341.000000,161341.000000,161341.000000
mean,85363.915979,25933.145457,0.464981,40415.308837
std,49278.047866,16192.292572,0.122027,25326.456157
min,1.000000,2736.000000,0.000000,5000.000000
25%,42687.000000,14828.000000,0.376793,25000.000000
50%,85367.000000,21076.000000,0.457566,35000.000000
75%,128057.000000,31768.000000,0.546254,50000.000000
max,170746.000000,200000.000000,0.976126,950000.000000


In [12]:
test_df.describe()

,client_id,credit_sum,score_shk,monthly_income
count,87130.000000,87130.000000,87130.000000,87130.000000
mean,216717.005509,25906.939924,0.464965,40340.503845
std,26538.609394,16146.947238,0.122416,24900.171004
min,170747.000000,3000.000000,0.000000,0.000000
25%,193762.250000,14808.000000,0.376901,25000.000000
50%,216695.500000,21018.000000,0.457525,35000.000000
75%,239717.750000,31689.000000,0.545748,50000.000000
max,262686.000000,200000.000000,0.974260,700000.000000


Выбросы и аномалии в числовых не категориальных данных отсутствуют. Следует рассмотреть категориальные данные.

In [13]:
for col in train_df.select_dtypes(include='category').columns:
    print(col, set(train_df[col]))

gender {'F', 'M'}
age {18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71}
marital_status {'UNM', 'DIV', 'WID', 'CIV', 'MAR'}
job_position {'INV', 'DIR', 'PNV', 'PNI', 'INP', 'PNA', 'WOI', 'UMN', 'BIS', 'NOR', 'WRK', 'WRP', 'PNS', 'ATP', 'SPC', 'HSK', 'BIU'}
credit_month {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 36}
tariff_id {1.6, 1.32, 1.0, 1.1, 1.5, 1.4, 1.7, 1.3, 1.16, 1.9, 1.17, 1.25, 1.43, 1.18, 1.26, 1.44, 1.19, 1.94, 1.52, 1.27, 1.2, 1.28, 1.21, 1.96, 1.29, 1.22, 1.23, 1.48, 1.56, 1.24, 1.91, 1.41}
education {'ACD', 'GRD', 'PGR', 'UGR', 'SCH'}
living_region {'ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ', 'ПЕРМСКИЙ КРАЙ', 'РЕСПУБЛИКА САХА', 'ХАНТЫ-МАНСИЙСКИЙ АО', 'РЕСП КАЛМЫКИЯ', 'ОБЛ КОСТРОМСКАЯ', 'ОБЛ ЛЕНИНГРАДСКАЯ', 'РЕСП ДАГЕСТАН', 'МОСКВА Г', 'ЕВРЕЙСКАЯ А

In [14]:
for col in test_df.select_dtypes(include='category').columns:
    print(col, set(test_df[col]))

gender {'F', 'M'}
age {18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71}
marital_status {'UNM', 'WID', 'DIV', 'CIV', 'MAR'}
job_position {'INV', 'DIR', 'PNV', 'PNI', 'INP', 'PNA', 'WOI', 'UMN', 'BIS', 'WRK', 'NOR', 'WRP', 'ONB', 'HSK', 'ATP', 'SPC', 'PNS', 'BIU'}
credit_month {3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 30, 31, 32, 36}
tariff_id {1.1, 1.32, 1.6, 1.4, 1.43, 1.44, 1.0, 1.9, 1.19, 1.5, 1.17, 1.25, 1.18, 1.26, 1.94, 1.27, 1.2, 1.7, 1.28, 1.21, 1.29, 1.22, 1.3, 1.23, 1.56, 1.48, 1.16, 1.24, 1.41, 1.91}
education {'ACD', 'GRD', 'PGR', 'UGR', 'SCH'}
living_region {'ЧУВАШСКАЯ РЕСПУБЛИКА - ЧУВАШИЯ', 'ПЕРМСКИЙ КРАЙ', 'ХАНТЫ-МАНСИЙСКИЙ АО', 'РЕСП КАЛМЫКИЯ', 'ОБЛ ЛЕНИНГРАДСКАЯ', 'ОБЛ КОСТРОМСКАЯ', 'РЕСП ДАГЕСТАН', 'МОСКВА Г', 'ЕВРЕЙСКАЯ АВТОНОМНАЯ', 'ОБЛ ВОЛГОГРАДСКАЯ',

В контексте препроцессинга существует интерес для детального анализа столбца "living_region", поскольку визуальный анализ показывает наличие одинаковых наименований областей с различным описанием.

* Необходимо привести наименование областей к стандартизированному виду.

## Region standartization

In [15]:
train_df.living_region.describe()

count             161341
unique               298
top       ОБЛ МОСКОВСКАЯ
freq               11371
Name: living_region, dtype: object

In [16]:
train_df.living_region.value_counts()

living_region
ОБЛ МОСКОВСКАЯ                   11371
КРАСНОДАРСКИЙ КРАЙ                6951
САНКТ-ПЕТЕРБУРГ                   5248
МОСКВА                            5077
ТАТАРСТАН РЕСП                    4918
                                 ...  
КРАЙ. КРАСНОЯРСКИЙ                   1
ПРИВОЛЖСКИЙ ФЕДЕРАЛЬНЫЙ ОКРУГ        1
ДАЛЬНИЙ ВОСТОК                       0
Г.ОДИНЦОВО МОСКОВСКАЯ ОБЛ            0
Г.МОСКВА                             0
Name: count, Length: 301, dtype: int64

Визуальный анализ показывает, что в столбце "living_region" количественно значимы как города в отдельности, так и области. Можно предположить, что клиенты проживающие в городах имеют кредитное отличие от клиентов проживающих в области. При этом клиенты проживающие в областях имеют некоторую кредитную схожесть с клиентами проживающими в экономических центрах областей, от клиентов иных областей и экономических центов. 

Выводы
* Города и области не должны быть обобщены;
* Идентификатор локации должен отражать географическую привязку, в виду экономической специфики РФ - наиболее экономически развитые локации распологаются недалеко от экономических центов.

Задачи:
* Найти идентификатор численно характеризующий географическую близость локаций.

Для выполнения данной задачи используется набор данных содержащий различные идентификаторы для областей и городов РФ.

Ссылка на используемый набор данных: https://github.com/hflabs/region/blob/master/region.csv

In [17]:
region_df=pd.read_csv(r'Data\Raw\region.csv')
region_df

,name,type,name_with_type,federal_district,kladr_id,fias_id,okato,oktmo,tax_office,postal_code,iso_code,timezone,geoname_code,geoname_id,geoname_name
0,Адыгея,Респ,Респ Адыгея,Южный,100000000000,d8327a56-80de-4df2-815c-4f6ab1224c50,79000000000,79000000,100,385000.0,RU-AD,UTC+3,RU.01,584222,Adygeya Republic
1,Башкортостан,Респ,Респ Башкортостан,Приволжский,200000000000,6f2cbfd8-692a-4ee4-9b16-067210bde3fc,80000000000,80000000,200,452000.0,RU-BA,UTC+5,RU.08,578853,Bashkortostan Republic
2,Бурятия,Респ,Респ Бурятия,Дальневосточный,300000000000,a84ebed3-153d-4ba9-8532-8bdf879e1f5a,81000000000,81000000,300,671000.0,RU-BU,UTC+8,RU.11,2050915,Buryatiya Republic
3,Алтай,Респ,Респ Алтай,Сибирский,400000000000,5c48611f-5de6-4771-9695-7e36a4e7529d,84000000000,84000000,400,649000.0,RU-AL,UTC+7,RU.03,1506272,Altai
4,Дагестан,Респ,Респ Дагестан,Северо-Кавказский,500000000000,0bb7fa19-736d-49cf-ad0e-9774c4dae09b,82000000000,82000000,500,368000.0,RU-DA,UTC+3,RU.17,567293,Dagestan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Чукотский,АО,Чукотский АО,Дальневосточный,8700000000000,f136159b-404a-4f1f-8d8d-d169e1374d5c,77000000000,77000000,8700,NaN,RU-CHU,UTC+12,RU.15,2126099,Chukotka
82,Ямало-Ненецкий,АО,Ямало-Ненецкий АО,Уральский,8900000000000,826fa834-3ee8-404f-bdbc-13a5221cfb6e,71140000000,71900000,8900,629000.0,RU-YAN,UTC+5,RU.87,1486462,Yamalo-Nenets
83,Крым,Респ,Респ Крым,Южный,9100000000000,bd8e6511-e4b9-4841-90de-6bbc231a789e,35000000000,35000000,9100,NaN,UA-43,UTC+3,UA.11,703883,Crimea
84,Севастополь,г,г Севастополь,Южный,9200000000000,6fdecb78-893a-4e3f-a5ba-aa062459463b,67000000000,67000000,9200,299700.0,UA-40,UTC+3,UA.20,694422,Sevastopol City


In [18]:
region_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              86 non-null     object 
 1   type              86 non-null     object 
 2   name_with_type    86 non-null     object 
 3   federal_district  85 non-null     object 
 4   kladr_id          86 non-null     int64  
 5   fias_id           86 non-null     object 
 6   okato             86 non-null     int64  
 7   oktmo             86 non-null     int64  
 8   tax_office        86 non-null     int64  
 9   postal_code       26 non-null     float64
 10  iso_code          86 non-null     object 
 11  timezone          86 non-null     object 
 12  geoname_code      86 non-null     object 
 13  geoname_id        86 non-null     int64  
 14  geoname_name      86 non-null     object 
dtypes: float64(1), int64(5), object(9)
memory usage: 10.2+ KB


### Index synthesis

In [19]:
region_df['iso_code']=region_df.iso_code.apply(lambda x: x[:2])
region_df=region_df.query('iso_code=="RU"').copy()
region_df.loc[:, 'geo_index']=list(region_df.tax_office.apply(lambda x: str(x)))
region_df.loc[:, 'timezone']=region_df.timezone.apply(lambda x: x[4:])
region_df.loc[:, 'geo_index']=region_df.timezone+region_df.geo_index
region_df=region_df.astype({"geo_index": 'int64'})
region_df=region_df[['name','geo_index']].copy()
region_df.reset_index(inplace=True, drop=True)
region_df

,name,geo_index
0,Адыгея,3100
1,Башкортостан,5200
2,Бурятия,8300
3,Алтай,7400
4,Дагестан,3500
...,...,...
78,Еврейская,107900
79,Ненецкий,32983
80,Ханты-Мансийский Автономный округ - Югра,58600
81,Чукотский,128700


In [20]:
region_df.sort_values(by=['geo_index'], inplace=True)
region_df.reset_index(inplace=True, drop=True)
region_df

,name,geo_index
0,Адыгея,3100
1,Дагестан,3500
2,Ингушетия,3600
3,Кабардино-Балкарская,3700
4,Калмыкия,3800
...,...,...
78,Еврейская,107900
79,Магаданская,114900
80,Сахалинская,116500
81,Камчатский,124100


In [21]:
region_df.to_csv(r'Data\Preprocessed\region.csv')

Ни один из индексов датасета не подходил для поставленной задачи. Для решения задачи был синтезирован обобщённый индекс, первая цифра каждого числа которого представляет собой временную зону UTC, а остальные индекс налогового офиса, из колонки "tax_office".

### Replace index

In [22]:
train_df['living_region']=train_df.living_region.apply(lambda x: re.sub(" +", " ", re.sub(r'[^\w\s]', ' ', x)))
train_df['living_region']=train_df['living_region'].str.lower()

In [23]:
region_df['name']=region_df.name.apply(lambda x: re.sub(" +", " ", re.sub(r'[^\w\s]', ' ', x)))
region_df['name']=region_df.name.apply(lambda x: x.split(" ")[0])
region_df['name']=region_df['name'].str.lower()

In [24]:
region_dict=dict()
for name, ind in zip(region_df.name, region_df.geo_index):
    region_dict[name]=ind

In [25]:
train_df['living_region'].replace(regex=region_dict, inplace=True)

In [26]:
train_df['living_region'].value_counts()[-20:]

living_region
114900                           150
3500                              65
пермская обл                      49
сев осетия алания                 47
128700                            28
32000                             24
горьковская обл                   19
3600                              17
читинская обл                     17
камчатская область                 3
россия                             2
приволжский федеральный округ      1
98                                 1
мытищинский р н                    1
москвоская обл                     1
эвенкийский ао                     1
гусь хрустальный р н               1
брянский                           1
орёл                               1
74                                 1
Name: count, dtype: int64

Некоторые не заменённые значения можно интерпретировать и заполнить вручную.

In [27]:
unidentified_regions_dict={'брянский': 33200, 'горьковская обл': 35200, 'гусь хрустальный р н': 33300, 'камчатская область': 124100, 
                           'москвоская обл': 35000, 'мытищинский р н': 35000, 'орёл': 35700, 'пермская обл': 55900, 'сев осетия алания': 31500, 
                           'читинская обл': 8300, 'эвенкийский ао': 72400}

In [28]:
train_df['living_region'].replace(regex=unidentified_regions_dict, inplace=True)

In [29]:
train_df['living_region'].value_counts()[-20:]

living_region
7400                             577
3900                             562
36000                            540
71700                            537
3100                             526
34400                            481
71900                            453
124100                           395
31500                            366
3800                             292
107900                           193
114900                           150
3500                              65
128700                            28
32000                             24
3600                              17
россия                             2
приволжский федеральный округ      1
98                                 1
74                                 1
Name: count, dtype: int64

Неинтерпретируемые значения подлежат удалению.

In [30]:
drop_list=['74', '98', 'приволжский федеральный округ', 'россия']

In [31]:
train_df=train_df[~train_df['living_region'].isin(drop_list)]
train_df.reset_index(inplace=True, drop=True)

In [33]:
train_df

,client_id,gender,age,marital_status,job_position,credit_sum,credit_month,tariff_id,score_shk,education,living_region,monthly_income,credit_count,overdue_credit_count,open_account_flg
0,1,M,48,MAR,UMN,59998.00,10,1.60,0.770249,GRD,32300,30000.0,1.0,1.0,0
1,2,F,28,MAR,UMN,10889.00,6,1.10,0.248514,GRD,37700,43000.0,2.0,0.0,0
2,3,M,32,MAR,SPC,10728.00,12,1.10,0.459589,SCH,46400,23000.0,5.0,0.0,0
3,4,F,27,DIV,SPC,12009.09,12,1.10,0.362536,GRD,33400,17000.0,2.0,0.0,0
4,5,M,45,MAR,SPC,16908.89,10,1.10,0.421385,SCH,57400,25000.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161331,170742,F,27,UNM,SPC,64867.00,12,1.10,0.535257,GRD,31600,40000.0,6.0,0.0,0
161332,170743,F,24,MAR,SPC,17640.00,6,1.60,0.573287,SCH,37800,30000.0,1.0,0.0,0
161333,170744,F,31,UNM,SPC,27556.47,10,1.32,0.416098,GRD,102500,40000.0,1.0,0.0,0
161334,170745,F,53,DIV,PNA,6189.00,12,1.10,0.482595,SCH,35800,31000.0,2.0,0.0,0


In [37]:
test_df['living_region']=test_df.living_region.apply(lambda x: re.sub(" +", " ", re.sub(r'[^\w\s]', ' ', x)))
test_df['living_region']=test_df['living_region'].str.lower()
test_df['living_region'].replace(regex=region_dict, inplace=True)
test_df['living_region'].replace(regex=unidentified_regions_dict, inplace=True)
test_df=test_df[~test_df['living_region'].isin(drop_list)]
test_df.reset_index(inplace=True, drop=True)

In [38]:
test_df['living_region'].value_counts()[-20:]

living_region
3700               338
33100              336
3900               334
71700              302
3100               299
36000              295
71900              266
34400              252
124100             211
31500              188
3800               144
107900             114
114900              78
3500                31
128700              26
32000               22
3600                 6
г челябинск          1
московский п         1
камчатс ий край      1
Name: count, dtype: int64

In [45]:
train_df['living_region'].replace(regex={"г челябинск": 57400, "камчатс ий край": 124100}, inplace=True)

In [46]:
test_df=test_df[~test_df['living_region'].isin(['московский п'])]
test_df.reset_index(inplace=True, drop=True)

In [47]:
train_df.to_csv(r'Data\Preprocessed\credit_train.csv')
test_df.to_csv(r'Data\Preprocessed\credit_test.csv')